# #BeastMode Playlist Bot
## Guaranteed to set your gym playlist and make you go Beast Mode
#### INFO 3510 -- Project 2
#### Jack Stein
<img src="playlistcover_50.jpg" alt="drawing" width="200"/>

### Step 1: Run the following 2 cells

In [ ]:
import os
from dotenv import load_dotenv
import discord
from discord.ext import commands
import pandas as pd
import requests
from time import sleep
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import base64
import discord
from discord.ext import commands
import nest_asyncio
nest_asyncio.apply()
from sklearn.preprocessing import MinMaxScaler 

In [ ]:
# Before you load in your .env file, create a new variable "SPOTIFY_USERNAME"
# If using Apple Music, instructions are below and you will have 
# to add other variables to your .env
load_dotenv("playlisttoken.env")

This playlist maker is designed for Spotify and Apple Music, each having a manual option and a Discord Bot option. The Apple manual section is designed to walk you though the steps assuming you have already done the Spotify one (it uses the same song and song recs as the Spotify method).

[Click here to jump down to Apple Music](#Apple-Music)

### Spotify

In [ ]:
# get the spotify id for a song I like
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyClientCredentials, SpotifyOAuth

#### For manual playlist creation run the following cells

In [ ]:
# code from INFO 5871 and 
# https://stackoverflow.com/questions/47379411/invalid-client-when-trying-to-use-spotipy-to-authorize-an-api-call
# this cell will generate the spotify token
manager = SpotifyClientCredentials(
    client_id = os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
)

sp = spotipy.Spotify(client_credentials_manager = manager)

username = os.getenv("SPOTIFY_USERNAME")
scope = 'user-library-read playlist-read-private playlist-read-collaborative playlist-modify-private playlist-modify-public ugc-image-upload'

token = util.prompt_for_user_token(username=username,client_id=os.getenv("SPOTIFY_CLIENT_ID"),
                                   client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"),
                                   redirect_uri='http://localhost:3000', 
                                   show_dialog=True,scope=scope)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [ ]:
# confirm that you are the correct user
sp.current_user()

In [ ]:
# create a base playlist for the songs to be added
user = sp.current_user()["id"]

# check if user has a playlist named "BeastModeBot". If not make a new playlist
num_user_playlists = len(sp.user_playlists(user)["items"])
playlist_found = False
for x in range(0,num_user_playlists):
    if sp.user_playlists(user)["items"][x]["name"] == "#BeastModeBot":
        playlist_found = True
        playlist = sp.user_playlists(user)["items"][x]
        
        
description_ = "Thank you for using the #BeastModeBot! This playlist has been specifically designed to help you power through your workout. Using Spotify's recommendation system we have generated a playlist based on your song choice. Then using our custome sorting algorithm we have designed the order of the playlist to help you keep pushing hard as you inevitably start getting tired. Have a great workout and go get after it!"

if playlist_found == False:
    playlist = sp.user_playlist_create(user=user, name="#BeastModeBot", public=True, description=description_)
    # make cover art for playlist
    # https://stackoverflow.com/questions/3715493/encoding-an-image-file-with-base64
    # Image made using DALL-E
    with open("playlistcover_50.jpg", "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
    sp.playlist_upload_cover_image(playlist_id=playlist["id"],image_b64=encoded_string)

In [ ]:
print("What song would you like a playlist created around?")
searchSong = input()
limit = 10
search = sp.search(searchSong, type="track", limit = limit)

# search for song
print(f"The following artists have a song by the name '{searchSong}'. Which one?")

# select correct artist
song_artists_returned = []
for x in range(0,limit):
    print("\t",search["tracks"]["items"][x]["artists"][0]["name"])
    song_artists_returned.append(search["tracks"]["items"][x]["artists"][0]["name"])
    
sleep(0.25)
artistName = input()

# verify if artist is an option
print(f"You wrote '{artistName}'")
if (artistName  in song_artists_returned):
    print("That artist is in the list")
else:
    while(artistName  not in song_artists_returned):  
        print("That artist is not an option, check the spelling and try again:")
        artistName = input()

# select API return that is for the correct artist
for x in range(0,limit):
    if (search["tracks"]["items"][x]["artists"][0]["name"] == artistName):
        song_rec_api_return = search["tracks"]["items"][x]
        break
# song_rec_api_return

# Get song ID
song_rec_id = song_rec_api_return["id"]
print(song_rec_id)

In [ ]:
# get the recomendations name's and ID's. All recs must have a tempo greater then the user's song
rec_tempo = sp.audio_features(song_rec_id)[0]["tempo"]
spot_recs = sp.recommendations(seed_tracks=[song_rec_id],min_tempo=rec_tempo,limit=20)
recs = {}
recs[song_rec_api_return["name"]] = song_rec_api_return["id"]
for x in range(0,len(spot_recs["tracks"])):
    if spot_recs["tracks"][x]["name"] not in recs:
        recs[spot_recs["tracks"][x]["name"]] = spot_recs["tracks"][x]["id"]
    sleep(.5)
# recs

In [ ]:
# add to a dataframe
df = pd.DataFrame(recs.items(), columns=["songName","id"])

# add ISRC for songs
def getISRCspotify(row):
    trackid = row["id"]
    trackreturn = sp.track(trackid)
    
    return trackreturn["external_ids"]["isrc"]

df["isrc"] = df.apply(getISRCspotify,axis=1)


# get audio featues and add back to the dataframe
features = sp.audio_features(df["id"].values)

df_ = pd.DataFrame(features)
df_.set_index('id',inplace=True)
df_ = df_.reset_index()
df = pd.merge(left = df, right = df_, how = "inner",left_on="id",right_on = "id")

In [ ]:
# Normalizing features
# https://www.geeksforgeeks.org/normalize-a-column-in-pandas/
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

# "MinMaxScaler doesn’t reduce the effect of outliers, but it linearily scales them down into a
# fixed range, where the largest occuring data point corresponds to the maximum value and 
# the smallest one corresponds to the minimum value"

# from sklearn.preprocessing import MinMaxScaler 
df_sklearn = df.copy() 
  
# apply normalization techniques 
columns = ["danceability","energy","loudness",'tempo']
for column in columns:
    df_sklearn[column+"_normal"] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1)) 
    
# sorting weights
# 30% danceablility, 40% tempo, 20% energy, 10% loudness
def score_sort_method(row):
    return ((row["danceability_normal"] * 0.3) + (row["tempo_normal"] * 0.4) + (row["energy_normal"] * 0.2) + (row["loudness_normal"] * 0.1))
    
df_sklearn['score'] = df_sklearn.apply(score_sort_method, axis=1)    
   
columns_normal = ["songName","id","uri","isrc","danceability_normal","energy_normal","loudness_normal","tempo_normal","score"]
df_sklearn_sorted = df_sklearn[columns_normal].sort_values("score").reset_index().drop("index",axis=1)

# graph
plt.figure(figsize=(12, 6))
columns = ["danceability_normal","energy_normal","loudness_normal","tempo_normal","score"]
rec_index = df_sklearn_sorted[df_sklearn_sorted['id'] == song_rec_id].index[0]
for column in columns:
    if column == "score":
        plt.plot(df_sklearn_sorted[column], label=column, linewidth=2.5, linestyle="--")
    else:
        plt.plot(df_sklearn_sorted[column], label=column)
    
    # mark the metrics of the rec song
    plt.scatter(rec_index, df_sklearn_sorted[column].iloc[rec_index], marker='o')

    
    
plt.title("Normalized Audio Features")
plt.legend()

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xticks.html
plt.xticks(ticks=range(0,len(df_sklearn_sorted)), labels=df_sklearn_sorted["songName"].values, rotation=90)
plt.show()

In [ ]:
# make graph as a figure - this is important for the discord bot to work
f,ax = plt.subplots(figsize=(12, 6))
f.set_facecolor('white')
columns = ["danceability_normal","energy_normal","loudness_normal","tempo_normal","score"]
rec_index = df_sklearn_sorted[df_sklearn_sorted['id'] == song_rec_id].index[0]
for column in columns:
    if column == "score":
        ax.plot(df_sklearn_sorted[column], label=column, linewidth=2.5, linestyle="--")
    else:
        ax.plot(df_sklearn_sorted[column], label=column)
    
    # mark the metrics of the rec song
    ax.scatter(rec_index, df_sklearn_sorted[column].iloc[rec_index], marker='o')#, label="Rec song")

    
    
ax.set_title("Normalized Audio Features")
ax.legend()
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xticks.html
ax.set_xticks(ticks=range(0, len(df_sklearn_sorted)))
ax.set_xticklabels(df_sklearn_sorted["songName"].values, rotation=90)

type(f.get_figure())

In [ ]:
# playlist runtime in minutes

total_minutes = df_sklearn["duration_ms"].sum() / 60000
hours = total_minutes // 60
minutes = total_minutes % 60
if hours == 1:
    print(f"Your playlist is {int(hours)} hour and {int(minutes)} minutes long")
else:
    print(f"Your playlist is {int(minutes)} minutes long")

In [ ]:
# making of the playlist

playlist_track_ids = df_sklearn_sorted["id"].values

sp.user_playlist_replace_tracks(user=user, playlist_id = playlist["id"], tracks = playlist_track_ids)

In [ ]:
# playlist URL to test if it's in your account

playlist["external_urls"]["spotify"]

#### For bot playlist creation, run cells here:

This bot is designed to that the command "!create song name" will trigger the bot to ask you questions and return your playlist back to you. Sometimes when giving the bot the artist, it doesn't work and you have to send the message again.

Note: This also requires "SPOTIFY_USERNAME" to be added to the .env file

In [ ]:
# turning it into a function for discord. not finished but used for testing

def createplaylist(searchSong):
    limit = 10
    search = sp.search(searchSong, type="track", limit = limit)

    # Search for song
    print(f"The following artists have a song by the name '{searchSong}'. Which one?")
    # Select correct artist
    song_artists_returned = []
    for x in range(0,limit):
        print("\t",search["tracks"]["items"][x]["artists"][0]["name"])
        print(type(search["tracks"]["items"][x]["artists"][0]["name"]))
        song_artists_returned.append(search["tracks"]["items"][x]["artists"][0]["name"])

    sleep(0.25)
    artistName = input()
    
# createplaylist("Spin Bout")

In [ ]:
# BOT
TOKEN = os.getenv("DISCORD_TOKEN")

In [ ]:
intents = discord.Intents().all()
bot = commands.Bot(command_prefix="!", intents = intents)

@bot.event
# @ sign signals a decorator
# decorator allows us to modify functions, methods, or classes
async def on_ready():
    print("{1} ({0}) has connected to Discord.".format(bot.user.id,bot.user.name))
# this code is saying "when the bot gets the event on_ready, the print statement will run"


@bot.command(name="create",help = "Create a playlist using the #BeastModeBot. Simply call the command followed by song name")
async def createPlaylist(ctx,*args):
    # need user infomation and check if they have playlist first.....
    
    manager = SpotifyClientCredentials(
        client_id = os.getenv("SPOTIFY_CLIENT_ID"),
        client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
    )

    sp = spotipy.Spotify(client_credentials_manager = manager)
    username = os.getenv("SPOTIFY_USERNAME")
    scope = 'user-library-read playlist-read-private playlist-read-collaborative playlist-modify-private playlist-modify-public ugc-image-upload'

    token = util.prompt_for_user_token(username,client_id=os.getenv("SPOTIFY_CLIENT_ID"),
                                       client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"),
                                       redirect_uri='http://localhost:3000', 
                                       show_dialog=True,scope=scope)
    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        await ctx.send(f"Can't get token for {username}")
        

    # create a base playlist for the songs to be added
    user = sp.current_user()["id"]

    # check if user has a playlist named "BeastModeBot". If not make a new playlist
    num_user_playlists = len(sp.user_playlists(user)["items"])
    playlist_found = False
    for x in range(0,num_user_playlists):
        if sp.user_playlists(user)["items"][x]["name"] == "#BeastModeBot":
            playlist_found = True
            playlist = sp.user_playlists(user)["items"][x]


    if playlist_found == False:
        description_ = "Thank you for using the #BeastModeBot! This playlist has been specifically designed to help you power through your workout. Using Spotify's recommendation system we have generated a playlist based on your song choice. Then using our custome sorting algorithm we have designed the order of the playlist to help you keep pushing hard as you inevitably start getting tired. Have a great workout and go get after it!"
        playlist = sp.user_playlist_create(user=user, name="#BeastModeBot", public=True, description=description_)
        # make cover art for playlist
        # https://stackoverflow.com/questions/3715493/encoding-an-image-file-with-base64
        # Image made using DALL-E
        with open("playlistcover_50.jpg", "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read())
        sp.playlist_upload_cover_image(playlist_id=playlist["id"],image_b64=encoded_string)
    #######################################
    
    
    limit = 10
    searchSong = " ".join(args)
    search = sp.search(searchSong, type="track", limit = limit)

    # search for song
    await ctx.send(f"The following 10 artists have a song by the name '{searchSong}':")
    # select correct artist
    song_artists_returned = []
    for x in range(0,limit):
        
        await ctx.send(f"\t {x+1}. "+search["tracks"]["items"][x]["artists"][0]["name"])
        song_artists_returned.append(search["tracks"]["items"][x]["artists"][0]["name"])

    await ctx.send("Which one?")
    msg = await bot.wait_for("message")
    artistName = msg.content
    # verify if artist is an option
    await ctx.send(f"You wrote '{artistName}'")
    if (artistName  in song_artists_returned):
        await ctx.send("That artist is in the list")
    else:
        while(artistName  not in song_artists_returned):  
            await ctx.send("That artist is not an option, check the spelling and try again:")
            msg = await bot.wait_for("message")
            artistName = msg.content
    
    # select API return that is for the correct artist
    for x in range(0,limit):
        if (search["tracks"]["items"][x]["artists"][0]["name"] == artistName):
            song_rec_api_return = search["tracks"]["items"][x]
            break
    await ctx.send(f"You have chosen to create a #BeastModeBot playlist around the song {song_rec_api_return['name']} by {artistName}")

    # get song ID
    song_rec_id = song_rec_api_return["id"]
    
    # get the recomendations name's and ID's. All recs must have a tempo greater then the user's song
    await ctx.send("Getting recomended songs...")
    rec_tempo = sp.audio_features(song_rec_id)[0]["tempo"]
    spot_recs = sp.recommendations(seed_tracks=[song_rec_id],min_tempo=rec_tempo,limit=20)
    recs = {}
    recs[song_rec_api_return["name"]] = song_rec_api_return["id"]
    for x in range(0,len(spot_recs["tracks"])):
        if spot_recs["tracks"][x]["name"] not in recs:
            recs[spot_recs["tracks"][x]["name"]] = spot_recs["tracks"][x]["id"]
        sleep(.5)
        
    # add to a dataframe
    await ctx.send("Found songs. Sorting... ")
    df = pd.DataFrame(recs.items(), columns=["songName","id"])
    # add ISRC for songs
    
    def getISRCspotify(row):
        trackid = row["id"]
        trackreturn = sp.track(trackid)

        return trackreturn["external_ids"]["isrc"]

    df["isrc"] = df.apply(getISRCspotify,axis=1)


    # get audio featues and add back to the dataframe
    features = sp.audio_features(df["id"].values)

    df_ = pd.DataFrame(features)
    df_.set_index('id',inplace=True)
    df_ = df_.reset_index()
    df = pd.merge(left = df, right = df_, how = "inner",left_on="id",right_on = "id")
    
    df_sklearn = df.copy() 
  
    # apply normalization techniques 
    columns = ["danceability","energy","loudness",'tempo']
    for column in columns:
        df_sklearn[column+"_normal"] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1)) 

    # sorting weights
    # 30% danceablility, 40% tempo, 20% energy, 10% loudness
    def score_sort_method(row):
        return ((row["danceability_normal"] * 0.3) + (row["tempo_normal"] * 0.4) + (row["energy_normal"] * 0.2) + (row["loudness_normal"] * 0.1))

    df_sklearn['score'] = df_sklearn.apply(score_sort_method, axis=1)    

    columns_normal = ["songName","id","uri","isrc","danceability_normal","energy_normal","loudness_normal","tempo_normal","score"]
    df_sklearn_sorted = df_sklearn[columns_normal].sort_values("score").reset_index().drop("index",axis=1)
    
    # graph
    f,ax = plt.subplots(figsize = (12, 10))
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots_adjust.html
    plt.subplots_adjust(bottom = 0.4)
    f.set_facecolor('white')
    columns = ["danceability_normal","energy_normal","loudness_normal","tempo_normal","score"]
    rec_index = df_sklearn_sorted[df_sklearn_sorted['id'] == song_rec_id].index[0]
    for column in columns:
        if column == "score":
            ax.plot(df_sklearn_sorted[column], label=column, linewidth=2.5, linestyle="--")
        else:
            ax.plot(df_sklearn_sorted[column], label=column)
        # mark the metrics of the rec song
        ax.scatter(rec_index, df_sklearn_sorted[column].iloc[rec_index], marker='o')
    ax.set_title("Normalized Audio Features")
    ax.legend()
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xticks.html
    ax.set_xticks(ticks=range(0, len(df_sklearn_sorted)))
    ax.set_xticklabels(df_sklearn_sorted["songName"].values, rotation=90)
    
    # send graph to discord
    f.savefig("playlist_line_graph.png")
    fileObj = discord.File("playlist_line_graph.png", filename="playlist_line_graph.png")
    await ctx.send("Graph of songs in playlist",file=fileObj)
    
    # playlist runtime
    total_minutes = df_sklearn["duration_ms"].sum() / 60000
    hours = total_minutes // 60
    minutes = total_minutes % 60
    if hours == 1:
        await ctx.send(f"Your playlist is {int(hours)} hour and {int(minutes)} minutes long")
    else:
        await ctx.send(f"Your playlist is {int(minutes)} minutes long")
    
    
    
    # make playlist
    playlist_track_ids = df_sklearn_sorted["id"].values
    sp.user_playlist_replace_tracks(user=user, playlist_id = playlist["id"], tracks = playlist_track_ids)
    await ctx.send(playlist["external_urls"]["spotify"])
    
    
    await ctx.send("DONE!") 
bot.run(TOKEN)

### Apple Music
The following cells are designed to create a playlist in Apple Music based on the same songs and sorting method as before. This also needed a few more keys added to the .env.

$\underline{Notes}$: The Apple Music API requires a few tokens to work, the first one  is based off having an Apple Developer account (you cannot make a playlist without this), another is based off having an Apple Music account. The Apple Music API also does not have a way to delete tracks or delete an existing playlist. Currently this program will add new songs to the bottom of the playlist if it already exists.

The interface for the Apple Bot looks exactly the same except outputs a statement everytime a song is added to the playlist rather then sending the link of the playlist 

Documentation:
https://developer.apple.com/documentation/applemusicapi/

In [ ]:
from datetime import datetime, timedelta
from time import time, mktime
import jwt
# Set tokens and unique ID's
# The SECRET KEY is unique to a users apple account. instructions on how to get your key can be found at https://github.com/therealmarius/Spotify-2-AppleMusic/blob/master/README.md 
# It also cannot be added to the .env since it has a very specific format
SECRET_KEY ="""
APPLE DEV KEY
"""
KEY_ID = os.getenv("APPLE_KEY_ID")
TEAM_ID = os.getenv("APPLE_TEAM_ID")
# music_key: instructions on how to get your key can be found at https://github.com/therealmarius/Spotify-2-AppleMusic/blob/master/README.md

#### The first section is for manual playlist creation

In [ ]:
# For this part, we will show how to 
# 1) take our df of spotify songs and find all the apple music ID for each song, save them back to the df
# 2) make a new playlist in the user Apple Music Library
# 3) upload songs to the new playlist

In [ ]:
# 1) take our df of spotify songs and find all the apple music ID for each song, save them back to the df
df_sklearn_sorted.head()

In [ ]:
# get an auth token for apple music api
# https://dev.to/hasone/generate-jwt-token-for-apple-store-connect-api-using-python-3j5h
secret_key = SECRET_KEY
key_id = KEY_ID
team_id = TEAM_ID

dt = datetime.now() + timedelta(minutes=19)
headers = {
    "alg": "ES256",
    "kid": key_id, 
    "typ": "JWT",
}
payload = {
    "iss": team_id,
    "iat": int(time()),
    "exp": int(mktime(dt.timetuple())),
    "aud": "appstoreconnect-v1",
}
signing_key = secret_key

# the key needs to be encoded in this specific way. The link above explains this
gen_jwt = jwt.encode(payload, signing_key, algorithm="ES256", headers=headers)

print(f"[JWT] {gen_jwt}")

In [ ]:
# search for single song
isrc = df_sklearn_sorted["isrc"].loc[0]

headers = {
    'Authorization': f'Bearer {gen_jwt}'
}
responce = requests.get(f'https://api.music.apple.com/v1/catalog/us/songs?filter[isrc]={isrc}',headers=headers).json()
# we will assume the first song returned is the correct song

# getting the ID
apple_song_id = responce["data"][0]["id"]
apple_song_id

In [ ]:
# test searching using a single song
url = f"https://api.music.apple.com/v1/catalog/us/songs/{apple_song_id}"
responce = requests.get(url,headers=headers).json()
responce["data"][0]["attributes"]["name"]

In [ ]:
def getAppleSongID(row):
    isrc = row["isrc"]
    headers = {
        'Authorization': f'Bearer {gen_jwt}'
    }
    responce = requests.get(f'https://api.music.apple.com/v1/catalog/us/songs?filter[isrc]={isrc}',headers=headers).json()
    if (responce["data"] == []):
        apple_song_id = np.nan
    else:
        apple_song_id = responce["data"][0]["id"]
    return apple_song_id

df_sklearn_sorted["apple_id"] = df_sklearn_sorted.apply(getAppleSongID,axis=1)
df_sklearn_sorted = df_sklearn_sorted.dropna(subset=['apple_id'])

In [ ]:
df_sklearn_sorted[["songName","apple_id"]].head(3)

In [ ]:
# 2) make a new playlist in the user Apple Music Library (or get the ID of the existing one)
# https://github.com/therealmarius/Spotify-2-AppleMusic/blob/master/convertsongs.py#L180

In [ ]:
# music_key: instructions on how to get your key can be found at https://github.com/therealmarius/Spotify-2-AppleMusic/blob/master/README.md
music_key = MUSIC_KEY

with requests.Session() as s:
        s.headers.update({"Authorization": f'Bearer {gen_jwt}',
                    "media-user-token": music_key})
        
playlist_name = "#BEASTMODEBOT"
description_ = "Thank you for using the #BeastModeBot! This playlist has been specifically designed to help you power through your workout. Using Spotify's recommendation system we have generated a playlist based on your song choice and added them to your Apple Music Library. Using our custome sorting algorithm we have designed the order of the playlist to help you keep pushing hard as you inevitably start getting tired. Have a great workout and go get after it!"
url = "https://api.music.apple.com/v1/me/library/playlists/"

data = {
    'attributes': {
        'name': playlist_name,
        'description': description_,
        'isPublic':True
    }
}
params = {
    "limit":100
}

def createPlaylist(s,url,data, params):
    apple_playlist_id = ""
    response = s.get(url,params=params)
    if (response.status_code == 200): # we successfully made the api get request
        for playlist in response.json()['data']:
#             print(playlist['attributes']['name'])
            if (playlist['attributes']['name'] == playlist_name):
                print(f"Playlist {playlist_name} already exists!")
                apple_playlist_id = playlist['id']
                return apple_playlist_id
    response = s.post(url, json=data) # playlist not found, we make it.
    if (response.status_code == 201): # making playlist worked
        print(f"Playlist {playlist_name} created!")
        apple_playlist_id = response.json()['data'][0]['id']
        return apple_playlist_id
    else: # failed to make playlist
        print("The API request to access your Library failed. Please check your auth tokens.")
        return apple_playlist_id
    
apple_playlist_id = createPlaylist(s,url,data,params)
apple_playlist_id

In [ ]:
# 3) upload songs to the new playlist

In [ ]:
for appleID in df_sklearn_sorted["apple_id"]:
    url = f"https://api.music.apple.com/v1/me/library/playlists/{apple_playlist_id}/tracks"
    json={"data":[{"id":f"{appleID}","type":"songs"}]}
    request = s.post(url,json=json)
    if requests.codes.ok:
        songName = df_sklearn_sorted.loc[df_sklearn_sorted['apple_id'] == appleID, 'songName'].iloc[0]
        print(f"Song {songName} added to playlist {playlist_name}!")

#### For playlist creation using the Discord Bot

This bot is designed to that the command "!createAM song name" will trigger the bot to ask you questions and return your playlist back to you. Sometimes when giving the bot the artist, it doesn't work and you have to send the message again.

In [ ]:
# BOT
TOKEN = os.getenv("DISCORD_TOKEN")
# APPLE KEYS
secret_key = SECRET_KEY
key_id = KEY_ID
team_id = TEAM_ID

dt = datetime.now() + timedelta(minutes=19)
headers = {
    "alg": "ES256",
    "kid": key_id, 
    "typ": "JWT",
}
payload = {
    "iss": team_id,
    "iat": int(time()),
    "exp": int(mktime(dt.timetuple())),
    "aud": "appstoreconnect-v1",
}
signing_key = secret_key

# the key needs to be encoded in this specific way. The link above explains this
gen_jwt = jwt.encode(payload, signing_key, algorithm="ES256", headers=headers)

In [ ]:
intents = discord.Intents().all()
bot = commands.Bot(command_prefix="!", intents = intents)

@bot.event
# @ sign signals a decorator
# decorator allows us to modify functions, methods, or classes
async def on_ready():
    print("{1} ({0}) has connected to Discord.".format(bot.user.id,bot.user.name))
# this code is saying "when the bot gets the event on_ready, the print statement will run"


# APPLE MUSIC BOT CREATION
@bot.command(name="createAM",help = "Create a playlist using the #BeastModeBot. Simply call the command followed by song name")
async def createPlaylist(ctx,*args):
    # need user infomation and check if they have playlist first.....
    
    manager = SpotifyClientCredentials(
        client_id = os.getenv("SPOTIFY_CLIENT_ID"),
        client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
    )

    sp = spotipy.Spotify(client_credentials_manager = manager)
    username = os.getenv("SPOTIFY_USERNAME")
    scope = 'user-library-read playlist-read-private playlist-read-collaborative playlist-modify-private playlist-modify-public ugc-image-upload'

    token = util.prompt_for_user_token(username,client_id=os.getenv("SPOTIFY_CLIENT_ID"),
                                       client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"),
                                       redirect_uri='http://localhost:3000', 
                                       show_dialog=True,scope=scope)
    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        await ctx.send(f"Can't get token for {username}")
    
    limit = 10
    searchSong = " ".join(args)
    search = sp.search(searchSong, type="track", limit = limit)

    # search for song
    await ctx.send(f"The following 10 artists have a song by the name '{searchSong}':")
    # select correct artist
    song_artists_returned = []
    for x in range(0,limit):
        
        await ctx.send(f"\t {x+1}. "+search["tracks"]["items"][x]["artists"][0]["name"])
        song_artists_returned.append(search["tracks"]["items"][x]["artists"][0]["name"])

    await ctx.send("Which one?")
    msg = await bot.wait_for("message")
    artistName = msg.content
    # verify if artist is an option
    await ctx.send(f"You wrote '{artistName}'")
    if (artistName  in song_artists_returned):
        await ctx.send("That artist is in the list")
    else:
        while(artistName  not in song_artists_returned):  
            await ctx.send("That artist is not an option, check the spelling and try again:")
            msg = await bot.wait_for("message")
            artistName = msg.content
    
    # select API return that is for the correct artist
    for x in range(0,limit):
        if (search["tracks"]["items"][x]["artists"][0]["name"] == artistName):
            song_rec_api_return = search["tracks"]["items"][x]
            break
    await ctx.send(f"You have chosen to create a #BeastModeBot playlist around the song {song_rec_api_return['name']} by {artistName}")

    # get song ID
    song_rec_id = song_rec_api_return["id"]
    
    # get the recomendations name's and ID's. All recs must have a tempo greater then the user's song
    await ctx.send("Getting recomended songs...")
    rec_tempo = sp.audio_features(song_rec_id)[0]["tempo"]
    spot_recs = sp.recommendations(seed_tracks=[song_rec_id],min_tempo=rec_tempo,limit=20)
    recs = {}
    recs[song_rec_api_return["name"]] = song_rec_api_return["id"]
    for x in range(0,len(spot_recs["tracks"])):
        if spot_recs["tracks"][x]["name"] not in recs:
            recs[spot_recs["tracks"][x]["name"]] = spot_recs["tracks"][x]["id"]
        sleep(.5)
        
    # add to a dataframe
    await ctx.send("Found songs. Sorting... ")
    df = pd.DataFrame(recs.items(), columns=["songName","id"])
    # add ISRC for songs
    
    def getISRCspotify(row):
        trackid = row["id"]
        trackreturn = sp.track(trackid)

        return trackreturn["external_ids"]["isrc"]

    df["isrc"] = df.apply(getISRCspotify,axis=1)


    # get audio featues and add back to the dataframe
    features = sp.audio_features(df["id"].values)

    df_ = pd.DataFrame(features)
    df_.set_index('id',inplace=True)
    df_ = df_.reset_index()
    df = pd.merge(left = df, right = df_, how = "inner",left_on="id",right_on = "id")
    
    df_sklearn = df.copy() 
  
    # apply normalization techniques 
    columns = ["danceability","energy","loudness",'tempo']
    for column in columns:
        df_sklearn[column+"_normal"] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1)) 

    # sorting weights
    # 30% danceablility, 40% tempo, 20% energy, 10% loudness
    def score_sort_method(row):
        return ((row["danceability_normal"] * 0.3) + (row["tempo_normal"] * 0.4) + (row["energy_normal"] * 0.2) + (row["loudness_normal"] * 0.1))

    df_sklearn['score'] = df_sklearn.apply(score_sort_method, axis=1)    

    columns_normal = ["songName","id","uri","isrc","danceability_normal","energy_normal","loudness_normal","tempo_normal","score"]
    df_sklearn_sorted = df_sklearn[columns_normal].sort_values("score").reset_index().drop("index",axis=1)
    
    # graph
    f,ax = plt.subplots(figsize = (12, 10))
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots_adjust.html
    plt.subplots_adjust(bottom = 0.4)
    f.set_facecolor('white')
    columns = ["danceability_normal","energy_normal","loudness_normal","tempo_normal","score"]
    rec_index = df_sklearn_sorted[df_sklearn_sorted['id'] == song_rec_id].index[0]
    for column in columns:
        if column == "score":
            ax.plot(df_sklearn_sorted[column], label=column, linewidth=2.5, linestyle="--")
        else:
            ax.plot(df_sklearn_sorted[column], label=column)
        # Mark the metrics of the rec song
        ax.scatter(rec_index, df_sklearn_sorted[column].iloc[rec_index], marker='o')
    ax.set_title("Normalized Audio Features")
    ax.legend()
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xticks.html
    ax.set_xticks(ticks=range(0, len(df_sklearn_sorted)))
    ax.set_xticklabels(df_sklearn_sorted["songName"].values, rotation=90)
    
    # send graph to discord
    f.savefig("playlist_line_graph.png")
    fileObj = discord.File("playlist_line_graph.png", filename="playlist_line_graph.png")
    await ctx.send("Graph of songs in playlist",file=fileObj)
    
    # playlist runtime
    total_minutes = df_sklearn["duration_ms"].sum() / 60000
    hours = total_minutes // 60
    minutes = total_minutes % 60
    if hours == 1:
        await ctx.send(f"Your playlist is {int(hours)} hour and {int(minutes)} minutes long")
    else:
        await ctx.send(f"Your playlist is {int(minutes)} minutes long")
    
    
    # get apple auth token
    await ctx.send(f"Making playlist on Apple Music")
    
    
    ##################
    
    # get apple song ids
    def getAppleSongID(row):
        isrc = row["isrc"]
        headers = {
            'Authorization': f'Bearer {gen_jwt}'
        }
        responce = requests.get(f'https://api.music.apple.com/v1/catalog/us/songs?filter[isrc]={isrc}',headers=headers).json()
        if (responce["data"] == []):
            apple_song_id = np.nan
        else:
            apple_song_id = responce["data"][0]["id"]
        return apple_song_id

    df_sklearn_sorted["apple_id"] = df_sklearn_sorted.apply(getAppleSongID,axis=1)
    df_sklearn_sorted = df_sklearn_sorted.dropna(subset=['apple_id'])

    
    # make playlist
    # music_key: instructions on how to get your key can be found at https://github.com/therealmarius/Spotify-2-AppleMusic/blob/master/README.md
    music_key = MUSIC_KEY

    with requests.Session() as s:
            s.headers.update({"Authorization": f'Bearer {gen_jwt}',
                        "media-user-token": music_key})

    playlist_name = "#BEASTMODEBOT"
    description_ = "Thank you for using the #BeastModeBot! This playlist has been specifically designed to help you power through your workout. Using Spotify's recommendation system we have generated a playlist based on your song choice. Then using our custome sorting algorithm we have designed the order of the playlist to help you keep pushing hard as you inevitably start getting tired. Have a great workout and go get after it!"
    url = "https://api.music.apple.com/v1/me/library/playlists/"

    data = {
        'attributes': {
            'name': playlist_name,
            'description': description_,
            'isPublic':True
        }
    }
    params = {
        "limit":100
    }

    def createPlaylist(s,url,data, params):
        apple_playlist_id = ""
        response = s.get(url,params=params)
        if (response.status_code == 200): # we successfully made the api get request
            for playlist in response.json()['data']:
                if (playlist['attributes']['name'] == playlist_name):
#                     ctx.send(f"Playlist {playlist_name} already exists!")
                    apple_playlist_id = playlist['id']
                    return apple_playlist_id
        response = s.post(url, json=data) # playlist not found, we make it.
        if (response.status_code == 201): # making playlist worked
#             ctx.send(f"Playlist {playlist_name} created!")
            apple_playlist_id = response.json()['data'][0]['id']
            return apple_playlist_id
        else: # failed to make playlist
            ctx.send("The API request to access your Library failed. Please check your auth tokens.")
            return apple_playlist_id

    apple_playlist_id = createPlaylist(s,url,data,params)
    
    # add songs to playlist
    
    for appleID in df_sklearn_sorted["apple_id"]:
        url = f"https://api.music.apple.com/v1/me/library/playlists/{apple_playlist_id}/tracks"
        json={"data":[{"id":f"{appleID}","type":"songs"}]}
        request = s.post(url,json=json)
        if requests.codes.ok:
            songName = df_sklearn_sorted.loc[df_sklearn_sorted['apple_id'] == appleID, 'songName'].iloc[0]
            await ctx.send(f"Song {songName} added to playlist {playlist_name}!") 
    
    
    await ctx.send("DONE!") 
bot.run(TOKEN)

Thank you!